Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:

1.Наименование вакансии.

2.Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).

3.Ссылку на саму вакансию.

4.Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с всех сайтов. Общий результат можно вывести с помощью dataFrame через pandas, сохранить в json, либо csv.

In [39]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
import pandas as pd

In [ ]:
User_Agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

In [84]:
def parsing_hhru(User_Agent, url):
    headers = {'User-Agent': User_Agent,}
    name_vacancy, link_vacancy, min_salary, max_salary, currency = [], [], [], [], []
    n = 0
    response = requests.get(url=url.format(n), headers=headers)
    while response.status_code != 404:
        response = requests.get(url=url.format(n), headers=headers)
        soup = bs(response.content, 'html.parser')
        list_vacancy = soup.find_all('a', {'class': "serp-item__title"})
        for i in range(len(list_vacancy)-1):
            name_vacancy.append(list_vacancy[i].getText())
            link_vacancy.append(list_vacancy[i]['href'])
            salary_data = list_vacancy[i].parent.parent.parent.find_all('span', {'data-qa': "vacancy-serp__vacancy-compensation"})
            if len(salary_data) == 0:
                min_salary.append(None)
                max_salary.append(None)
                currency.append(None)
            else:
                salary_data = salary_data[0].get_text().replace('\u202f', '').split()

                if len(salary_data) == 3 and salary_data[0] == 'от':
                    min_salary.append(int(salary_data[1]))
                    max_salary.append(None)
                    currency.append(salary_data[2])

                if len(salary_data) == 3 and salary_data[0] == 'до':
                    min_salary.append(None)
                    max_salary.append(int(salary_data[1]))
                    currency.append(salary_data[2])

                if len(salary_data) == 4:
                    min_salary.append(int(salary_data[0]))
                    max_salary.append(int(salary_data[2]))
                    currency.append(salary_data[3])
        n = n+1
        sleep(5)       
    vacancy_from_hh = pd.DataFrame(data = {'name_vacancy' : list(name_vacancy), 'link_vacancy': list(link_vacancy), 
                                           'min_salary' : list(min_salary), 'max_salary' : list(max_salary), 
                                          'currency' : list(currency)})
    vacancy_from_hh['source'] = 'https://hh.ru/' 
    return vacancy_from_hh
                                

In [85]:
url = 'https://hh.ru/search/vacancy?text=python&from=suggest_post&area=1&customDomain=1&page={}&hhtmFrom=vacancy_search_list'
vacancy_from_hh = parsing_hhru(User_Agent = User_Agent, url = url)

In [129]:
vacancy_from_hh.head()

,name_vacancy,link_vacancy,min_salary,max_salary,currency,source
0,Python developer (Machine Learning Engineer),https://hh.ru/vacancy/72074865?from=vacancy_se...,NaN,NaN,None,https://hh.ru/
1,Python developer (Machine Learning Engineer),https://hh.ru/vacancy/73468878?from=vacancy_se...,NaN,NaN,None,https://hh.ru/
2,Дата-инженер / Data Engineer,https://hh.ru/vacancy/71236005?from=vacancy_se...,130000.0,NaN,руб.,https://hh.ru/
3,Старший дата-инженер / Senior Data Engineer,https://hh.ru/vacancy/72161518?from=vacancy_se...,270000.0,NaN,руб.,https://hh.ru/
4,Middle/Senior Data Engineer в Sber AI Lab,https://hh.ru/vacancy/72868606?from=vacancy_se...,NaN,NaN,None,https://hh.ru/


In [150]:
def parsing_superjob(User_Agent, url):
    headers = {'User-Agent': User_Agent,}
    name_vacancy, link_vacancy, min_salary, max_salary, currency = [], [], [], [], []
    n = 0
    response = requests.get(url=url.format(n), headers=headers)
    soup = bs(response.content, 'html.parser')
    list_vacancy = soup.find_all('span', {'class': "_2s70W _31udi _7mW5l _17ECX _1B2ot _3EXZS _3pAka ofdOE"})    
    while len(list_vacancy) != 0:
        response = requests.get(url=url.format(n), headers=headers)
        soup = bs(response.content, 'html.parser')
        list_vacancy = soup.find_all('span', {'class': "_2s70W _31udi _7mW5l _17ECX _1B2ot _3EXZS _3pAka ofdOE"})
        for i in range(len(list_vacancy)-1):
            name_vacancy.append(list_vacancy[i].getText())
            link_one = "https://www.superjob.ru" + list_vacancy[0].findChildren("a" , recursive=False)[0]['href']
            link_vacancy.append(link_one)
            salary_data = soup.find_all('div', {'class': "f-test-text-company-item-salary"})[i].getText()
            if salary_data == 'По договорённости':
                min_salary.append(None)
                max_salary.append(None)
                currency.append(None)
            else:
                salary_data = salary_data.replace('\u202f', '').split()

                if len(salary_data) == 3:
                    min_salary.append(int(salary_data[0]+salary_data[1]))
                    max_salary.append(int(salary_data[0]+salary_data[1]))
                    currency.append(salary_data[2])
                    
                if len(salary_data) == 4 and salary_data[0] == 'от':
                    min_salary.append(int(salary_data[1]+salary_data[2]))
                    max_salary.append(None)
                    currency.append(salary_data[3])                   

                if len(salary_data) == 4 and salary_data[0] == 'до':
                    min_salary.append(None)
                    max_salary.append(int(salary_data[1]+salary_data[2]))
                    currency.append(salary_data[3])

                if len(salary_data) == 6:
                    min_salary.append(int(salary_data[0]+salary_data[1]))
                    max_salary.append(int(salary_data[3]+salary_data[4]))
                    currency.append(salary_data[5])
        n = n+1
        sleep(5)       
    vacancy_from_superjob = pd.DataFrame(data = {'name_vacancy' : list(name_vacancy), 'link_vacancy': list(link_vacancy), 
                                           'min_salary' : list(min_salary), 'max_salary' : list(max_salary), 
                                          'currency' : list(currency)})
    vacancy_from_superjob['source'] = 'https://www.superjob.ru/' 
    return vacancy_from_superjob

In [151]:
url = 'https://www.superjob.ru/vacancy/search/?keywords=python&geo%5Bt%5D%5B0%5D=4&page={}'
vacancy_from_superjob = parsing_superjob(User_Agent = User_Agent, url = url)

In [153]:
vacancy_from_superjob.head()

,name_vacancy,link_vacancy,min_salary,max_salary,currency,source
0,Разработчик Python,https://www.superjob.ru/vakansii/razrabotchik-...,180000.0,NaN,₽/месяц,https://www.superjob.ru/
1,Python разработчик (Middle),https://www.superjob.ru/vakansii/razrabotchik-...,200000.0,260000.0,₽/месяц,https://www.superjob.ru/
2,Python-разработчик с функционалом data-аналитика,https://www.superjob.ru/vakansii/razrabotchik-...,NaN,NaN,None,https://www.superjob.ru/
3,Senior Python Developer (remote),https://www.superjob.ru/vakansii/razrabotchik-...,NaN,NaN,None,https://www.superjob.ru/
4,Python Backend Developer (Team Lead/Remote),https://www.superjob.ru/vakansii/razrabotchik-...,NaN,250000.0,₽/месяц,https://www.superjob.ru/


In [155]:
vacancy_all = pd.concat([vacancy_from_hh,vacancy_from_superjob])

In [161]:
vacancy_all.to_csv('vacancy_all.csv',index=False, encoding='utf-16')